In [2]:
import os
import sys
sys.path.append("..")
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pandas as pd
from utils import *
import string
from collections import Counter
import spacy
import nltk
from nltk.tokenize import word_tokenize
import importlib.metadata as importlib_metadata
importlib_metadata.version('tqdm')
import spacy
from spacy import Language
nlp = spacy.load("de_core_news_sm")
import sqldf

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jannis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('tagsets')

[nltk_data] Downloading package tagsets to /Users/jannis/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jannis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/jannis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package tagsets to /Users/jannis/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

## Actual Code to do initial POS-tagging!

In [5]:
df = pd.read_csv('../data/data_files/all_sentences_class_by_links.csv')

In [7]:
df['text'] = df['text'].astype(str)

In [8]:
words_rem = df.text.map(remove_stopwords)
punct_rem = words_rem.map(lambda x:x.translate(str.maketrans('','',string.punctuation)))
pos_list=[]
for sentence in punct_rem:
    sentence_pos = get_sentence_pos_list(sentence,nlp)
    pos_list.append(sentence_pos)
pos_seqings = list(map(lambda x: ' '.join(x),pos_list))
df['pos_seq'] = pos_seqings
df['pos_list'] = pos_list

In [9]:
df.to_csv('../data/data_files/all_sentences_linklabeled_pos.csv')

## Determining the POS-Union between True / False 

In [10]:
df = pd.read_csv('../data/data_files/all_sentences_linklabeled_pos.csv')

In [11]:
# define a custom aggregation function to calculate the pos_union value
def pos_union_func(x):
    # count the number of unique labels for this pos_seq
    label_count = len(set(x['label']))
    # if there is only one unique label, return True
    if label_count == 1:
        return False
    if label_count == 2:
        return True
    

# group by pos_seq and apply the custom aggregation function to calculate pos_union
pos_union = df.groupby('pos_seq').apply(pos_union_func).reset_index(name='pos_union')

# merge the pos_union back into the original dataframe
df_union = df.merge(pos_union, on='pos_seq', how='left')

# print the resulting dataframe
print(df_union)

        Unnamed: 0.1  Unnamed: 0  \
0                  0           0   
1                  1           1   
2                  2           2   
3                  3           3   
4                  4           4   
...              ...         ...   
460872        460872         677   
460873        460873         678   
460874        460874         679   
460875        460875         680   
460876        460876         681   

                                                     text  label  \
0       Rudolf Walter Richard Heß  (* 2 April   1894  ...   True   
1       Heß war ab 1933 Reichsminister ohne Geschäftsb...   True   
2       Öffentlich tat sich Heß als fanatischer Anhäng...   True   
3       1933 ernannte ihn Adolf Hitler   zu seinem Ste...   True   
4       Am 1 Mai 1941 flog Heß in das Vereinigte König...   True   
...                                                   ...    ...   
460872  Dieser Missionar macht geltend, daß durch die ...   True   
460873  Die Operation w

In [14]:
df_union.value_counts('label')

label
False    237936
True     222941
dtype: int64

In [15]:
df_union.value_counts('pos_union')

pos_union
False    376785
True      83884
dtype: int64

In [16]:
df_union.head()

,Unnamed: 0.1,Unnamed: 0,text,label,pos_seq,pos_list,pos_union
0,0,0,Rudolf Walter Richard Heß (* 2 April 1894 ...,True,PROPN PROPN PROPN PROPN SPACE NUM NOUN NUM PRO...,"['PROPN', 'PROPN', 'PROPN', 'PROPN', 'SPACE', ...",False
1,1,1,Heß war ab 1933 Reichsminister ohne Geschäftsb...,True,PROPN ADP NUM NOUN ADV ADP NUM NOUN ADJ NOUN,"['PROPN', 'ADP', 'NUM', 'NOUN', 'ADV', 'ADP', ...",False
2,2,2,Öffentlich tat sich Heß als fanatischer Anhäng...,True,ADV VERB PROPN ADJ NOUN NOUN ADV,"['ADV', 'VERB', 'PROPN', 'ADJ', 'NOUN', 'NOUN'...",False
3,3,3,1933 ernannte ihn Adolf Hitler zu seinem Ste...,True,NUM VERB PROPN PROPN ADJ NOUN,"['NUM', 'VERB', 'PROPN', 'PROPN', 'ADJ', 'NOUN']",True
4,4,4,Am 1 Mai 1941 flog Heß in das Vereinigte König...,True,NUM NOUN NUM VERB PROPN ADJ ADV SPACE ADJ NOUN...,"['NUM', 'NOUN', 'NUM', 'VERB', 'PROPN', 'ADJ',...",False


In [17]:
df_union.to_csv('../data/data_files/all_sentences_linklabeled_pos_unions.csv')

## Filtering for non-overlapping

In [18]:
filtered_df = df_union.query('not (label == False and pos_union == True)')

In [20]:
filtered_df.value_counts('label')

label
True     222941
False    181565
dtype: int64

In [21]:
filtered_df.value_counts('pos_union')

pos_union
False    376785
True      27513
dtype: int64

In [ ]:
filtered_df.to_csv('../data/data_files/all_sentences_posfilter_linklabeled.csv')

## Preprocess for other stuff

In [ ]:
regex = r"[^A-Za-z0-9\s'':;.?()\[\]@\\\-/\$´€`&]+"
df_nan = df_union.copy()
df_nan['text'] = df_union['text'].fillna('')
df_nan = df_nan.reset_index(drop=True)
df_filtered = df_nan[~df_nan['text'].str.contains(regex)]


In [ ]:
# group by pos_seq and calculate the union of labels
pos_union = df.groupby('pos_seq')['label'].apply(set).apply(lambda x: all(x)).reset_index()

# merge the pos_union back into the original dataframe
df = df.merge(pos_union, on='pos_seq', how='left')

# rename the newly added column to pos_union
df = df.rename(columns={'label_x': 'label', 'label_y': 'pos_union'})

# print the resulting dataframe
print(df)

In [ ]:
df_queried.head()

In [ ]:
union_counter

In [ ]:
query = """SELECT
  text,
  pos_seq,
  label,
  (SELECT COUNT(DISTINCT label) FROM df WHERE pos_seq=pos_seq)   AS pos_union
FROM
  df
GROUP BY
  text, pos_seq
"""
df_union = sqldf.run(query)

In [ ]:
print(df_union.value_counts('pos_union'))

In [ ]:
# find pos_seq values with both True and False labels
pos_seqs = df.groupby('pos_seq')['label'].apply(lambda x: set(x.unique()))
mixed_pos_seqs = pos_seqs[pos_seqs.apply(lambda x: True if len(x) > 1 else False)].index

# filter dataframe to only include rows with mixed pos_seq values
mixed_df = df[df['pos_seq'].isin(mixed_pos_seqs)]

# group by pos_seq and filter to only include groups with both True and False labels
filtered_groups = mixed_df.groupby('pos_seq').filter(lambda x: set(x['label'].unique()) == set([True, False]))

# print out the filtered DataFrame
print(filtered_groups)


In [ ]:
# identify pos_seq values with at least one True and one False target
mixed_pos_seqs = df.groupby('pos_seq')['label'].apply(lambda x: set(x.unique())).\
                 apply(lambda x: True if len(x) > 1 and False in x else False).\
                 loc[lambda x: x == True].index

# filter original DataFrame to keep only rows with mixed pos_seq values
mixed_df = df[df['pos_seq'].isin(mixed_pos_seqs)]

# identify rows with target=True
true_rows = mixed_df[mixed_df['target'] == True]

# identify pos_seq values with at least one True row
true_pos_seqs = true_rows['pos_seq'].unique()

# filter original DataFrame to remove rows with target=False where a True row exists with the same pos_seq value
filtered_df = df[~((df['target'] == False) & (df['pos_seq'].isin(true_pos_seqs)))]

# print out the filtered DataFrame
print(filtered_df)


In [ ]:
# group the dataframe by 'pos_seq'
grouped = df.groupby('pos_seq')

# loop through each group
for pos_seq, group in grouped:
    # check if there is at least one 'True' and one 'False' value in the 'label' column
    if group['label'].isin([True, False]).all():
        # print out the rows with matching 'pos_seq' values
        print(group)

In [ ]:
df.value_counts('pos_seq')[25:50000]

In [ ]:
def get_sentence_pos_seq(sentence:str,nlp:Language)->str:
    """Returns the part-of-speech tags of a given sentence as concatenated strings.

    Args:
        sentence (str): A sentence, preferably cleaned of stopwords & punctuation.
        nlp (Language): NLTK language object.

    Returns:
        str: String of POS tag sequence.
    """
    doc = nlp(sentence)
    sentence_pos =""
    for token in doc:
        sentence_pos=sentence_pos+" "+token.pos_
    return sentence_pos

In [ ]:

all_articles = pd.read_csv('../data/data_files/articles_w_fulltext_standard.csv', sep=',')[['title', 'bytes', 'full_text']]
i = 1
df = preprocess_classify_wiki_text(all_articles['full_text'].iloc[0])
while i < len(all_articles):
    working_df = preprocess_classify_wiki_text(all_articles['full_text'].iloc[i])
    df = pd.concat([df, working_df])
    i += 1


In [ ]:

other_pos_list=[]
for i in punct_rem:
    a = get_sentence_pos_list(i,nlp)
    other_pos_list.append(a)


In [ ]:
df = pd.read_csv('../data/data_files/all_sentences_w_pos_union.csv')


In [ ]:
# Create a boolean mask to select rows where 'label' is TRUE and 'pos_union' is FALSE
mask = (df['label'] == True) & (df['pos_union'] == False)

# Use boolean indexing to select the rows that match the mask
filtered_df = df.loc[mask]


In [3]:
df_filtered = df[df.label == True OR (df.label == FALSE && df.pos_union == TRUE)]

In [ ]:
def get_sentence_pos_list(sentence,nlp:Language)->list[str]:
    """Returns the part-of-speech tags of a given sentence as list.

    Args:
        sentence (str): A sentence, preferably cleaned of stopwords & punctuation.
        nlp (Language): NLTK language object.

    Returns:
        list[str]: List of String POS tags.
    """
    
    doc = nlp(sentence)
    sentence_pos = []
    for token in doc:
        sentence_pos.append(str(token.pos_))
    return sentence_pos


In [ ]:
pos_list=[]
for sentence in punct_rem:
    
    a = get_sentence_pos(sentence)
    pos_list.append(a)
df["pos"] = pos_list

In [ ]:
df["pos"] = pos_list

We need to add remove Numbers & Remove Punctuation here!!

In [ ]:
punct_rem

Iterates over each sentence, applies nlp individually and then adds up string of POS tags of a sentence, than appends it to list. TAKE ABOUT 9 Minutes!

In [ ]:
df["pos"] = pos_list

In [ ]:
claim_pos_list=list(df[df["label"]==True]["pos"])
noclaim_pos_list=list(df[df["label"]==False]["pos"])

In [ ]:
dataframe_claim=(df[df["label"]==True])
dataframe_noclaim=(df[df["label"]==False])

Gives most frequent pos combis

In [ ]:
dataframe_noclaim["pos"].value_counts()

In [ ]:
print(dataframe_noclaim["pos"].value_counts())
print(dataframe_claim["pos"].value_counts())


iterrates throguh noclaim list, and prints out every sentence, whiches pos is matched with any pos of claims

In [ ]:
i=len(noclaim_pos_list)
j=0
while(j<i):
    if(noclaim_pos_list[j] in claim_pos_list):
        print(dataframe_noclaim["text"].iloc[j])
    j=j+1

In [ ]:
print(df["text"].iloc[1:5])
sentences = list(df["text"].iloc[1:5])
paraphrases = util.paraphrase_mining(model,sentences)
for paraphrase in paraphrases:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))


In [ ]:
sentences = list(df["text"].iloc[1:5])
embeddings = model.encode(sentences)
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")
